# Wildfire Data Pre-Processing

#### Data Source:

Monitoring Trends in Burn Severity (MTBS)

Burned Areas Boundaries Dataset (1984-2022)

https://www.mtbs.gov/direct-download

In [1]:
import pandas as pd
import numpy as np

## Load Data From MTBS

In [2]:
import geopandas as gdp

In [3]:
df_raw = gdp.read_file('data/mtbs/mtbs_perims_DD.shp')

In [4]:
df = df_raw.copy()

## Drop Unneeded Columns

In [5]:
df = df.drop(columns=[
    'Map_ID',
    'Map_Prog',
    'Asmnt_Type',
    'Pre_ID',
    'Post_ID',
    'Perim_ID',
    'dNBR_offst',
    'dNBR_stdDv',
    'NoData_T',
    'IncGreen_T',
    'Low_T',
    'Mod_T',
    'High_T',
    'Comment'
])

## Rename Columns

In [6]:
df = df.rename(columns={
            'Event_ID': 'mtbs_id',
            'irwinID': 'irwin_id',
            'Incid_Name': 'incident_name',
            'Incid_Type': 'incident_type',
            'BurnBndAc': 'burned_acreage',
            'Ig_Date': 'ignition_date',
            'BurnBndLat': 'latitude',
            'BurnBndLon': 'longitude'
})

## Add a Year Column

In [7]:
df['year'] = df.ignition_date.apply(lambda date: date[:4])

## (Optional) Add a state column

In [8]:
"""This works to compute the state for each fire,
but it will take 3+ hours to work for all 30k fires
import reverse_geocoder
import tqdm
tqdm.pandas()

df = df.astype({
    'latitude': np.float64,
    'longitude': np.float64
})

def get_state(row):
    response = reverse_geocoder.search((row.latitude, row.longitude))
    return response[0]['admin1']
df['state'] = df.progress_apply(get_state, axis=1)
""";

## Data Cleanliness

The data, as provided by MTBS, only contains missing values for IRWIN IDs. This is to be expected, as IRWIN IDs were introduced in 2015 and this dataset goes back to 1984. Otherwise, the dataset has already been cleaned by MTBS.

In [9]:
df.isna().sum()

mtbs_id               0
irwin_id          25937
incident_name         0
incident_type         0
burned_acreage        0
latitude              0
longitude             0
ignition_date         0
geometry              0
year                  0
dtype: int64

## Save Data

In [10]:
df.to_file('data/mtbs.geojson', driver='GeoJSON')  